In [1]:
# for creating a responsive plot 
%matplotlib inline
%matplotlib widget 

import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import threading
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
finger_joints_names = [
    "WRIST",
    "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP",
    "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP",
    "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP",
    "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP",
    "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP"
]

In [4]:
def convert_to_lower_left_coord(landmarks):
    converted_landmarks = []
    #wrist_landmark = landmarks[finger_joints_names.index("WRIST")]
    for landmark in landmarks:
        x = landmark.x 
        y = 1 - landmark.y
        z = landmark.z
        converted_landmarks.append({"x": landmark.x, "y": y, "z": landmark.z})
    return converted_landmarks

In [5]:
def vector_length(v):
    return math.sqrt(v[0] ** 2 + v[1] ** 2 + v[2] ** 2)

In [6]:
def cross_product(a, b):
    c = [a[1]*b[2] - a[2]*b[1],
         a[2]*b[0] - a[0]*b[2],
         a[0]*b[1] - a[1]*b[0]]
    return c

In [7]:
def angle_between(a, b):
    dot_product = sum(x * y for x, y in zip(a, b))
    magnitude_a = math.sqrt(sum(x**2 for x in a))
    magnitude_b = math.sqrt(sum(x**2 for x in b))
    cos_theta = dot_product / (magnitude_a * magnitude_b)
    cos_theta = min(1, cos_theta)
    angle = math.acos(cos_theta)
    return math.degrees(angle)

In [8]:
def normalize_vector(v):
    magnitude = math.sqrt(sum(x**2 for x in v))
    return [x / magnitude for x in v]

In [9]:
def tuple_to_matrix(x, y, z, t):
    array = np.array([x, y, z, t])
    array = np.transpose(array)
    array = np.concatenate([array, np.array([[0, 0, 0, 1]])], axis=0)
    return array

In [10]:
def plot_3d(x, y, z):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(x, y, z, c='b', marker='o')
    
    # Draw a line
    def draw_line(ax, prev_p, next_p):
        ax.plot([prev_p[0], next_p[0]], [prev_p[1], next_p[1]], [prev_p[2], next_p[2]], c='b')
        return ax

    for i in range(x.shape[0]):
        if i == 0:
            continue
        if (i - 1) % 4 == 0:
            prev_p = [0, 0, 0]
        else:
            prev_p = next_p
        next_p = [x[i], y[i], z[i]]
        ax = draw_line(ax, prev_p, next_p)

    # Draw Oxyz coord
    ax.plot([0, 1], [0, 0], [0, 0], c='r')
    ax.plot([0, 0], [0, 1], [0, 0], c='g')
    ax.plot([0, 0], [0, 0], [0, 1], c='b')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    # Set the limits for each axis
    ax.set_xlim(0, 0.5)
    ax.set_ylim(0, 0.5)
    ax.set_zlim(0, 0.5)  # Setting custom limits for z-axis

    fig.savefig('view_1.png')

    ax.view_init(elev=11, azim=-2)
    fig.savefig('view_2.png')
    

    ax.view_init(elev=90, azim=-90)
    fig.savefig('view_3.png')
    
    view_1 = cv2.imread('view_1.png')
    view_2 = cv2.imread('view_2.png')
    view_3 = cv2.imread('view_3.png')
    
    cv2.imshow("View 1", view_1)
    cv2.moveWindow("View 1", 2650, 0)
    
    cv2.imshow("View 2", view_2)
    cv2.moveWindow("View 2", 2000, 550)
    
    cv2.imshow("View 3", view_3)
    cv2.moveWindow("View 3", 2650, 550)

    plt.close()
    #return ax

In [11]:
def convert_to_wrist_coordinate(landmarks, R_inv):
    wrist_landmarks = []
    for i in range(len(landmarks)):
        landmark = landmarks[i]
        j_camera = np.array([landmark["x"], landmark["y"], landmark["z"], 1])
        j_wrist = np.dot(R_inv, j_camera)
        wrist_landmarks.append(j_wrist)
    return wrist_landmarks

In [12]:
def hand_tracking(image):
    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    wrist_landmarks = None
        
    results = hands.process(processed_image)
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))
            """
            for i, landmarks in enumerate(hand.landmark):
                #if landmarks.z < 0:
                    #print(finger_joints_names[i])
                    #print(landmarks)
                if i not in [8, 12]:
                    continue
                coords = tuple(np.multiply(np.array((landmarks.x, 
                                                     landmarks.y)), 
                                        [640, 480]).astype(int))  
                image = cv2.putText(image, str(round(landmarks.z, 2)), coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
            """
            landmarks = convert_to_lower_left_coord(hand.landmark)
            u = (landmarks[finger_joints_names.index("INDEX_FINGER_MCP")]["x"] - landmarks[finger_joints_names.index("WRIST")]["x"],
                 landmarks[finger_joints_names.index("INDEX_FINGER_MCP")]["y"] - landmarks[finger_joints_names.index("WRIST")]["y"],
                 landmarks[finger_joints_names.index("INDEX_FINGER_MCP")]["z"] - landmarks[finger_joints_names.index("WRIST")]["z"])

            y = (landmarks[finger_joints_names.index("MIDDLE_FINGER_MCP")]["x"] - landmarks[finger_joints_names.index("WRIST")]["x"],
                 landmarks[finger_joints_names.index("MIDDLE_FINGER_MCP")]["y"] - landmarks[finger_joints_names.index("WRIST")]["y"],
                 landmarks[finger_joints_names.index("MIDDLE_FINGER_MCP")]["z"] - landmarks[finger_joints_names.index("WRIST")]["z"])
            
            x = cross_product(y, u)
            z = cross_product(x, y)
            x, y, z = normalize_vector(x), normalize_vector(y), normalize_vector(z)

            #print('debug angle: ', angle_between(y_old, y))

            #print('|x|: ', vector_length(x))
            #print('|y|: ', vector_length(y))
            #print('|z|: ', vector_length(z))
            #print("(x,z) = ", angle_between(x, z))
            #print("(x,y) = ", angle_between(x, y))
            #print("(z, y) = ", angle_between(z, y))

            #print("x: ", x)
            #print("y: ", y)
            #print("z: ", z)

            w_c = (landmarks[finger_joints_names.index("WRIST")]["x"], 
                   landmarks[finger_joints_names.index("WRIST")]["y"],
                   landmarks[finger_joints_names.index("WRIST")]["z"])

            R = tuple_to_matrix(x, y, z, w_c)
            R_inv = np.linalg.inv(R)

            o_wrist = np.int32(np.dot(R_inv, np.array([w_c[0], w_c[1], w_c[2], 1])))
            
            #print(o_wrist)

            wrist_landmarks = convert_to_wrist_coordinate(landmarks, R_inv)
            wrist_landmarks = np.array(wrist_landmarks)
            #print("wrist_landmarks", wrist_landmarks)
            
            plot_3d(wrist_landmarks[:, 0], wrist_landmarks[:, 1], wrist_landmarks[:, 2]) 

            #print(wrist_landmarks)
            #x = np.dot(R_inv, np.array([x[0], x[1], x[2], 1]))  # To ensure our computations are correct, this one must equal to [1, 0, 0]
            #y = np.dot(R_inv, np.array([y[0], y[1], y[2], 1]))  # To ensure our computations are correct, this one must equal to [0, 1, 0]
            #z = np.dot(R_inv, np.array([z[0], z[1], z[2], 1]))  # To ensure our computations are correct, this one must equal to [0, 0, 1]
            
            #print('x_w: ', x)
            #print('y_w: ', y)
            #print('z_w: ', z)
            #print(test_2)
            """
            wrist_coords = wrist_landmarks[0, :3]
            index_finger_coords = wrist_landmarks[5:9, :3]

            angles = []
            for i in range(index_finger_coords.shape[0]):
                if i == 0:
                    continue
                if (i - 1) % 4 == 0:
                    prev_p = [0, 0, 0]
                else:
                    prev_p = next_p
                next_p = [x[i], y[i], z[i]]
                ax = draw_line(ax, prev_p, next_p)"""

            angle_1 = angle_between(wrist_landmarks[6, :3] - wrist_landmarks[5, :3], wrist_landmarks[5, :3])
            coords = tuple(np.multiply(np.array((hand.landmark[5].x, 
                                                 hand.landmark[5].y)), 
                                        [640, 480]).astype(int))  
            image = cv2.putText(image, str(round(angle_1, 2)), coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
            print(angle_1)
            
    return image, wrist_landmarks

In [13]:
cap = cv2.VideoCapture(0)    

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        # Flip on horizontal
        image = cv2.flip(frame, 1)
        print(image.shape)

        # Set flag
        #image.flags.writeable = False

        image, wrist_landmarks = hand_tracking(image.copy())
        cv2.imshow("Hand Tracking", image)
        cv2.moveWindow("Hand Tracking", 2000, 0)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@0.330] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@0.330] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
I0000 00:00:1715588384.748282   53001 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1715588384.751209   53056 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) UHD Graphics 630 (CFL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [14]:
wrist_landmarks

NameError: name 'wrist_landmarks' is not defined

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot points
ax.scatter(wrist_landmarks[:, 0], wrist_landmarks[:, 1], wrist_landmarks[:, 2], c='b', marker='o')

# Draw a line
def draw_line(ax, prev_p, next_p):
    ax.plot([prev_p[0], next_p[0]], [prev_p[1], next_p[1]], [prev_p[2], next_p[2]], c='b')
    return ax

for i in range(wrist_landmarks.shape[0]):
    if i == 0:
        continue
    if (i - 1) % 4 == 0:
        prev_p = [0, 0, 0]
    else:
        prev_p = next_p
    next_p = [wrist_landmarks[i, 0], wrist_landmarks[i, 1], wrist_landmarks[i, 2]]
    ax = draw_line(ax, prev_p, next_p)

    # Draw Oxyz coord
ax.plot([0, 1], [0, 0], [0, 0], c='r')
ax.plot([0, 0], [0, 1], [0, 0], c='g')
ax.plot([0, 0], [0, 0], [0, 1], c='b')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Set the limits for each axis
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(0, 0.5)
ax.set_zlim(-0.5, 0.5)  # Setting custom limits for z-axis

ax.view_init(elev=11, azim=-2)


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot points
ax.scatter(wrist_landmarks[:, 0], wrist_landmarks[:, 1], wrist_landmarks[:, 2], c='b', marker='o')

# Draw a line
def draw_line(ax, prev_p, next_p):
    ax.plot([prev_p[0], next_p[0]], [prev_p[1], next_p[1]], [prev_p[2], next_p[2]], c='b')
    return ax

for i in range(wrist_landmarks.shape[0]):
    if i == 0:
        continue
    if (i - 1) % 4 == 0:
        prev_p = [0, 0, 0]
    else:
        prev_p = next_p
    next_p = [wrist_landmarks[i, 0], wrist_landmarks[i, 1], wrist_landmarks[i, 2]]
    ax = draw_line(ax, prev_p, next_p)

    # Draw Oxyz coord
ax.plot([0, 1], [0, 0], [0, 0], c='r')
ax.plot([0, 0], [0, 1], [0, 0], c='g')
ax.plot([0, 0], [0, 0], [0, 1], c='b')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Set the limits for each axis
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(0, 0.5)
ax.set_zlim(-0.5, 0.5)  # Setting custom limits for z-axis

ax.view_init(elev=90, azim=-90)

# RealSense

In [12]:
def hand_tracking_realsense(image):
    #processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    processed_image = image
    wrist_landmarks = None
        
    results = hands.process(processed_image)
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))
            
            landmarks = convert_to_lower_left_coord(hand.landmark)
            u = (landmarks[finger_joints_names.index("INDEX_FINGER_MCP")]["x"] - landmarks[finger_joints_names.index("WRIST")]["x"],
                 landmarks[finger_joints_names.index("INDEX_FINGER_MCP")]["y"] - landmarks[finger_joints_names.index("WRIST")]["y"],
                 landmarks[finger_joints_names.index("INDEX_FINGER_MCP")]["z"] - landmarks[finger_joints_names.index("WRIST")]["z"])

            y = (landmarks[finger_joints_names.index("MIDDLE_FINGER_MCP")]["x"] - landmarks[finger_joints_names.index("WRIST")]["x"],
                 landmarks[finger_joints_names.index("MIDDLE_FINGER_MCP")]["y"] - landmarks[finger_joints_names.index("WRIST")]["y"],
                 landmarks[finger_joints_names.index("MIDDLE_FINGER_MCP")]["z"] - landmarks[finger_joints_names.index("WRIST")]["z"])
            
            x = cross_product(y, u)
            z = cross_product(x, y)
            x, y, z = normalize_vector(x), normalize_vector(y), normalize_vector(z)

            w_c = (landmarks[finger_joints_names.index("WRIST")]["x"], 
                   landmarks[finger_joints_names.index("WRIST")]["y"],
                   landmarks[finger_joints_names.index("WRIST")]["z"])

            R = tuple_to_matrix(x, y, z, w_c)
            R_inv = np.linalg.inv(R)

            o_wrist = np.int32(np.dot(R_inv, np.array([w_c[0], w_c[1], w_c[2], 1])))
            
            wrist_landmarks = convert_to_wrist_coordinate(landmarks, R_inv)
            wrist_landmarks = np.array(wrist_landmarks)
            
            plot_3d(wrist_landmarks[:, 0], wrist_landmarks[:, 1], wrist_landmarks[:, 2]) 

            """
            wrist_coords = wrist_landmarks[0, :3]
            index_finger_coords = wrist_landmarks[5:9, :3]

            angles = []
            for i in range(index_finger_coords.shape[0]):
                if i == 0:
                    continue
                if (i - 1) % 4 == 0:
                    prev_p = [0, 0, 0]
                else:
                    prev_p = next_p
                next_p = [x[i], y[i], z[i]]
                ax = draw_line(ax, prev_p, next_p)"""

            angle_1 = angle_between(wrist_landmarks[6, :3] - wrist_landmarks[5, :3], wrist_landmarks[5, :3])
            coords = tuple(np.multiply(np.array((hand.landmark[5].x, 
                                                 hand.landmark[5].y)), 
                                        [640, 480]).astype(int))  
            image = cv2.putText(image, str(round(angle_1, 2)), coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
            print(angle_1)
            
    return image, wrist_landmarks

In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

try:
    while True:

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        # Convert images to numpy arrays
        image = np.asanyarray(color_frame.get_data())

            #image, wrist_landmarks = hand_tracking_realsense(image.copy())

        # Show images
        cv2.namedWindow('Hand Tracking', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()

In [22]:
x = np.random.rand(5, 2, 2)

In [23]:
x

array([[[0.30441226, 0.25626777],
        [0.48686008, 0.86273574]],

       [[0.27199172, 0.19041019],
        [0.06588798, 0.84560219]],

       [[0.35100584, 0.04572388],
        [0.48384535, 0.4281388 ]],

       [[0.96426854, 0.78286143],
        [0.46291566, 0.47570115]],

       [[0.18594956, 0.76290807],
        [0.92146523, 0.20808616]]])

In [27]:
directions = np.sign(np.linalg.det(x))

In [28]:
directions

array([ 1.,  1.,  1.,  1., -1.])

In [29]:
y = np.random.rand(5)

In [30]:
y * directions

array([ 0.19541859,  0.24095602,  0.25926387,  0.42549541, -0.69043752])

In [31]:
A.shape = (4, 3)
B.shape = (4, 3)

# Remove 0 value we have
A.shape = (4, 2)
B.shape = (4, 2)

# 4 is 4 fingers, 2 value in lies in a plan
M.shape = (4, 2, 2)

# Get sign of determinant
direction.shape = (4,)

ValueError: cannot reshape array of size 2 into shape (4,3)

In [32]:
# remove all zero
# turn to M with 3 dims
# get sign of det

In [6]:
import numpy as np

A = np.random.rand(10, 2)
B = np.random.rand(10, 2)

In [12]:
A

array([[0.66356393, 0.82445232],
       [0.82060082, 0.87741292],
       [0.05043704, 0.52706274],
       [0.56251808, 0.60015203],
       [0.89173084, 0.79523052],
       [0.58667804, 0.61369583],
       [0.00941407, 0.33200859],
       [0.68677854, 0.75942193],
       [0.30653357, 0.94177531],
       [0.18102917, 0.82445096]])

In [13]:
B

array([[0.60186584, 0.84899647],
       [0.63841814, 0.36616631],
       [0.35445971, 0.78492193],
       [0.5384292 , 0.77482165],
       [0.7695046 , 0.86398979],
       [0.09912928, 0.51797188],
       [0.88303986, 0.6870676 ],
       [0.42919355, 0.95212344],
       [0.95597035, 0.18110174],
       [0.67446968, 0.13337354]])

In [16]:
C = np.concatenate([A[:, None, :], B[:, None, :]], axis=1)

In [17]:
np.linalg.det(C)

array([ 0.06715374, -0.25967995, -0.14723337,  0.11271181,  0.1585128 ,
        0.2430475 , -0.28670872,  0.32795896, -0.8447955 , -0.53192268])

In [20]:
np.sign(np.linalg.det(C))

array([ 1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.])

In [24]:
np.linalg.det(np.array([[0, 1], [1, 0]]))

-1.0